Ссылка на данный блокнот: https://colab.research.google.com/drive/1QJhmaAGopzt9t7oh9CV420mWCQAGD63i?usp=drive_link

Чтобы в Google Collab работал PySpark необходимо установить на виртуальное окружение пакет Java, скачать архив Spark, распаковать его и для Python установить библиотеку findspark.


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

Добавляем значения "JAVA_HOME" и "SPARK_HOME" в переменные среды с помощью метода "os.environ"   и указываем каталоги, где находятся Java и пакет Spark. Объявляем "Master" в нашем случае он локальный ("local[*]").


In [ ]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
import findspark

findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)  # Property used to format output tables better
spark

Устанавливаем и подключаем необходимые для выполнения задания библиотеки, описанные в README.md и подключаем Google Drive, чтобы сохранять сгенерированные файлы локально.

In [ ]:
!pip install pdoc3
!pip install country_list
!pip install countryinfo

# TODO: порядок импортов: сначала либы питона в алфавитном порядке(import, потом from)
# посмотри на datetime, их можно прописать в одной строке, т.к. импорты из одной области
# потом либы установленные в том же порядке
# после импортов 2 пустые строки
import hashlib
import logging
import random
import string
from datetime import date, datetime, timedelta
from uuid import uuid4

import country_list
import countryinfo
import pyspark.sql.functions as F
import pyspark.sql.types as T
from google.colab import drive


drive.mount("/content/gdrive")

logger = logging.getLogger()
logging.basicConfig(
    filename = "mylog.log",
    format = "%(asctime)s - %(levelname)s - %(funcName)s: %(lineno)d - %(message)s",
    datefmt='%H:%M:%S',
)

In [ ]:
countries = list(country_list.countries_for_language("en"))

Функция генерирующее число в строковом виде.

In [ ]:
# TODO: также хороший код style писать тип возвращаемого значения
# TODO: generate_value - 1) нужен глагол, как в описании, т.к. метод что-то именно делает + value - слишком обезличенно
def generate_digits(count: int = 10) -> str:
    """Генерирует число по заданному количеству цифр.

    Parameters:
        count: int
            Количество цифр в числе.

    Returns:
        str:
            Сгенерированное число.
    """

    return "".join(random.choices(string.digits, k=count))

Создаём функцию "generate_rows_table", которая позволит сгенерировать список необходимых данных представленных для дальнейшего заполнения таблиц(витрин) из файла "Витрины.docx". Аргументами данной функции является число строк, которые мы хотим вставить в таблицу за раз и дата в поле "timestampcolumn".

In [ ]:
def generate_rows_table(
    count: int = 1,
    timestampcolumn: datetime = date.today()
) -> list:
    """Генерует строки таблицы.

    Parameters:
        count: int
            Количество строк.
        timestampcolumn: datetime
            Дата партиции.

    Returns:
        list_cookies: list
            Возвращает массив списков, представляющий собой строку таблицы.
    """
    
    # TODO: полезно разделять пустыми строками разные логические части (инициализация переменных и цикл/начало if else, когда кажется,
    # что в глазах все склеивается)
    list_cookies = list()
    
    for _ in range(count):
        # генерация cookies
        inn = generate_digits(12)

        _sa_cookie_a = {
            "key": "_sa_cookie_a",
            "value": f"SA1.{uuid4()}.{generate_digits(10)}"
        }

        _fa_cookie_a = {
            "key": "_fa_cookie_a",
            "value": f"ads2.{generate_digits(10)}.{generate_digits(10)}"
        }

        # TODO: когда поле состоит только из одного вызова, то джойн уже не нужен, т.к. из метода и так строка выходит
        _ym_cookie_c = {
            "key": "_ym_cookie_c",
            "value": generate_digits(20)
        }

        _fbp = {
            "key": "_fbp",
            "value": f"fb.{random.choice(string.digits)}."
                     f"{generate_digits(13)}."
                     f"{generate_digits(9)}"
        }

        org_uid = {
            "key": "org_uid",
            "value": generate_digits(7)
        }

        user_uid = {
            "key": "user_uid",
            "value": generate_digits(7)
        }

        # TODO: по сути можем сразу схлопнуть 2 и 3 знака
        user_phone = {
            "key": "user_phone",
            "value": f"79{generate_digits(9)}"
        }

        user_mail = {
            "key": "user_mail",
            "value": f"""{''.join(random.choices(string.ascii_letters +
                             string.digits, k=10))}@user.io"""
        }

        # генерация event_type
        event_type = random.choice(["SUBMIT", "REGISTER", "SUBMIT_MD5"])

        # генерация event_action
        event_action = random.choice(["pageview", "event", "login-check-otp"])

        # генерация data_value
        if event_type == "SUBMIT":
            data_value = hashlib.sha256(bytes(inn, encoding="utf-8")).hexdigest()
        elif event_type == "SUBMIT_MD5":
            data_value = hashlib.md5(bytes(inn, encoding="utf-8")).hexdigest()
        else:
            data_value = None

        # генерация страный, города и геопозиции
        while True:
            try:
                # TODO: лучше не завязываться на индексах, тут мы скипаем первый элемент и берем только второй
                _, geocountry = random.choice(countries)
                country = countryinfo.CountryInfo(geocountry)
                city = country.capital()
                geoaltitude = ",".join(map(str, country.latlng()))
                break
            except KeyError as e:
                logger.error(f"В списке 'country' отсутствует значение {str(e)}")

        # Генерация meta_platform
        meta_platform = random.choice(["WEB", "MOBAIL"])

        # Генерация user_os
        if meta_platform == "WEB":
            user_os = random.choice(["Mac", "Windows", "Ubuntu"])
        else:
            user_os = random.choice(["IOS", "Android", "HarmonyOS", "BlackBerryOS"])
        # Генерация systemlanguage
        systemlanguage = random.choice(["RU", "ENG"])

        # Генерация screensize
        screensize = "1920x1080"

        # Добавление спосков в единый список
        list_cookies.append([
            inn,
            [
                _sa_cookie_a,
                _fa_cookie_a,
                _ym_cookie_c,
                _fbp,
                org_uid,
                user_uid,
                user_phone,
                user_mail
            ],
            event_type,
            event_action,
            data_value,
            geocountry,
            city,
            user_os,
            systemlanguage,
            geoaltitude,
            meta_platform,
            screensize,
            timestampcolumn
        ])
    return list_cookies

Объявление схемы данных таблицы

In [ ]:
# схема данных
schema = T.StructType([
    T.StructField("inn", T.StringType(), True),
    T.StructField(
        "raw_cookie",
        T.ArrayType(
            T.MapType(
                T.StringType(),
                T.StringType()
            )
        )
    ),
    T.StructField("event_type", T.StringType(), True),
    T.StructField("event_action", T.StringType(), True),
    T.StructField("data_value", T.StringType(), True),
    T.StructField("geocountry", T.StringType(), True),
    T.StructField("city", T.StringType(), True),
    T.StructField("user_os", T.StringType(), True),
    T.StructField("systemlanguage", T.StringType(), True),
    T.StructField("geoaltitude", T.StringType(), True),
    T.StructField("meta_platform", T.StringType(), True),
    T.StructField("screensize", T.StringType(), True),
    T.StructField("timestampcolumn", T.DateType(), True)
])

Цикл, генерируемый записи таблицы и сохраняемый их в файлы формата JSON, в
каталоги по дням недели. Все каталоги создаются автоматически. Данный цикл эмулирует постоянное добавление файлов. Необходимо запустить в фоновом режиме, пока идет работа с файлом "Create_data_marts.ipynb". Это требуется, чтобы смоделировать приближенную картину к реальности при работе с витриной "B" и витриной "D", так как они одинаковые, но количество строк в них может быть разным.

Чтобы не допустить переполнение диска, цикл отработает 10 раз после чего
выключится самостоятельно. По итогу будет сгеренировано тестовых данных за 10 дней.



In [ ]:
# Цикл генерирует каталоги с файлами в формате JSON согласно партиционирования
# TODO: тут опять же про пустые строки, с ними гораздо проще читается
current_date = date.today()
folder = "/content/gdrive/MyDrive/data/json/"

if not os.path.isdir(folder):
    os.makedirs(folder)
    
i = 0

while i < 10:
    for _ in range(random.randrange(5, 10)):
        df = spark.createDataFrame(
            generate_rows_table(30, current_date),
            schema=schema
        )
        
        df.coalesce(1).write.mode("append") \
          .json(f"{folder}{current_date.strftime('%Y_%m_%d')}")

    # TODO: есть операторы += -= *= и т.д., чтобы не дублировать левую переменную
    current_date += timedelta(1)
    i = i + 1

In [ ]:
# TODO: всегда останавливаем спарк сессию
spark.stop()